# Training WordPiece tokenizer example

In [78]:
import datasets
from tokenizers import Tokenizer, models, normalizers, pre_tokenizers, processors, trainers

## Loading Vietnamese dataset

In [79]:
vi_dataset = datasets.load_dataset("facebook/belebele", "vie_Latn", split="test")
vi_dataset

Dataset({
    features: ['link', 'question_number', 'flores_passage', 'question', 'mc_answer1', 'mc_answer2', 'mc_answer3', 'mc_answer4', 'correct_answer_num', 'dialect', 'ds'],
    num_rows: 900
})

In [80]:
vi_dataset[0]

{'link': 'https://en.wikibooks.org/wiki/Accordion/Right_hand',
 'question_number': 1,
 'flores_passage': 'Hãy đảm bảo tay của bạn được thư giãn nhất có thể khi nhấn tất cả các nốt nhạc chính xác - và cố gắng đừng làm nhiều cử động ngón tay không cần thiết. Chơi đàn theo cách này sẽ giúp bạn giảm mất sức tối đa. Hãy nhớ rằng, bạn không cần phải dùng nhiều lực để ấn phím nhằm tăng âm lượng như khi chơi đàn piano. Trên đàn phong cầm, để tăng âm lượng, bạn sử dụng ống thổi tác động mạnh hơn hoặc nhanh hơn.',
 'question': 'Theo đoạn văn, đâu không được coi là lời khuyên đúng để chơi được đàn phong cầm?',
 'mc_answer1': 'Để tăng âm lượng, dùng thêm nhiều lực khi ấn phím',
 'mc_answer2': 'Hạn chế tối thiểu động tác thừa để giữ sức',
 'mc_answer3': 'Chú ý khi nhấn các nốt nhạc phải giữ cho tay được thư giãn nhất có thể',
 'mc_answer4': 'Tăng tốc độ di chuyển hộp xếp để tăng âm lượng',
 'correct_answer_num': '1',
 'dialect': 'vie_Latn',
 'ds': '2023-06-01'}

In [81]:
sample_text = vi_dataset[0]["flores_passage"]
sample_text

'Hãy đảm bảo tay của bạn được thư giãn nhất có thể khi nhấn tất cả các nốt nhạc chính xác - và cố gắng đừng làm nhiều cử động ngón tay không cần thiết. Chơi đàn theo cách này sẽ giúp bạn giảm mất sức tối đa. Hãy nhớ rằng, bạn không cần phải dùng nhiều lực để ấn phím nhằm tăng âm lượng như khi chơi đàn piano. Trên đàn phong cầm, để tăng âm lượng, bạn sử dụng ống thổi tác động mạnh hơn hoặc nhanh hơn.'

## Training WordPiece

### Init tokenizer

In [82]:
from tokenizers.models import WordPiece

wp_tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))

### Normalization

In [83]:
normalizer = normalizers.Sequence([normalizers.NFD(), 
                                normalizers.Lowercase()])
wp_tokenizer.normalizer = normalizer

In [84]:
normalized_sample_text = normalizer.normalize_str(sample_text)
normalized_sample_text

'hãy đảm bảo tay của bạn được thư giãn nhất có thể khi nhấn tất cả các nốt nhạc chính xác - và cố gắng đừng làm nhiều cử động ngón tay không cần thiết. chơi đàn theo cách này sẽ giúp bạn giảm mất sức tối đa. hãy nhớ rằng, bạn không cần phải dùng nhiều lực để ấn phím nhằm tăng âm lượng như khi chơi đàn piano. trên đàn phong cầm, để tăng âm lượng, bạn sử dụng ống thổi tác động mạnh hơn hoặc nhanh hơn.'

### Pre_tokenizer

In [85]:
pre_tokenizer = pre_tokenizers.Sequence([pre_tokenizers.Whitespace(), 
                                        pre_tokenizers.Digits(individual_digits=True)])
wp_tokenizer.pre_tokenizer = pre_tokenizer

In [86]:
pre_tokenized_sample_text = pre_tokenizer.pre_tokenize_str(normalized_sample_text)
pre_tokenized_sample_text[:5]

[('hãy', (0, 4)),
 ('đảm', (5, 9)),
 ('bảo', (10, 14)),
 ('tay', (15, 18)),
 ('của', (19, 23))]

### Train

In [87]:
trainer = trainers.WordPieceTrainer(vocab_size=30522, 
                                    special_tokens=["<BLANK>", "<S>", "<UNK>"],
                                    show_progress=True, 
                                    continuing_subword_prefix="_")

In [88]:
def batch_iterator(batch_size=1000):
    for i in range(0, len(vi_dataset), batch_size):
        yield vi_dataset[i : i + batch_size]["flores_passage"]
        
wp_tokenizer.train_from_iterator(batch_iterator(), trainer=trainer, length=len(vi_dataset))

In [89]:
wp_tokenizer.get_vocab_size()

6207

In [90]:
wp_tokenizer.get_vocab()

{'istanbul': 3493,
 '_housie': 5348,
 'ống': 958,
 'gosling': 3487,
 'galileo': 5109,
 'anh': 513,
 '_ởi': 496,
 '_onia': 3180,
 'trư': 1431,
 'thám': 1831,
 'abu': 3694,
 'pattave': 5185,
 'vai': 1291,
 'ogarzynska': 5824,
 '_dern': 4202,
 'rhine': 6156,
 'holo': 5961,
 'cớ': 4607,
 'domingo': 5672,
 'phá': 253,
 '_ào': 405,
 'danh': 915,
 'sharipov': 5737,
 'trấn': 1501,
 'chuôi': 4480,
 '_thru': 6035,
 'cường': 2861,
 'pr': 2763,
 'zou': 4094,
 'report': 4858,
 '_mal': 4235,
 'ft': 3772,
 'sezen': 4979,
 ').': 702,
 'eskim': 3411,
 '_sl': 2528,
 'lũ': 2024,
 'jesus': 3847,
 'sullivan': 6126,
 'mìn': 3909,
 'tục': 1069,
 '_agan': 4816,
 '_aqu': 4114,
 'tại': 319,
 '_pus': 4179,
 'iphone': 5491,
 '_op': 1606,
 'ngon': 2577,
 '.""': 1080,
 'paget': 6109,
 'zambia': 3530,
 'nghiệm': 828,
 '_pson': 3066,
 '_ình': 219,
 '_agascar': 4819,
 'mario': 4929,
 'ayatollah': 5417,
 '_ất': 188,
 'cẩu': 2864,
 '_un': 1121,
 'thắng': 976,
 '_isha': 4623,
 'tộc': 1132,


In [91]:
wp_tokenizer.save('vi-wiki-tokenizer/word-piece-tokenizer.json')

## Pos-processor

In [92]:
from tokenizers.processors import TemplateProcessing
tokenizer = Tokenizer.from_file('vi-wiki-tokenizer/word-piece-tokenizer.json')
tokenizer.post_processor = TemplateProcessing(
    single="[CLS] $A [SEP]",
    pair="[CLS] $A [SEP] $B:1 [SEP]:1",
    special_tokens=[("[CLS]", 1), ("[SEP]", 2)],
)

In [93]:
tokenizer.get_vocab_size()

6207

## Encode

In [ ]:
sample_encoded = tokenizer.encode(sample_text)
sample_encoded.ids

## Decode

In [95]:
tokenizer.decode(sample_encoded.ids)

'hãy đảm bảo tay của bạn được thư giãn nhất có thể khi nhấn tất cả các nốt nhạc chính xác - và cố gắng đừng làm nhiều cử động ngón tay không cần thiết . chơi đàn theo cách này sẽ giúp bạn giảm mất sức tối đa . hãy nhớ rằng , bạn không cần phải dùng nhiều lực để ấn phím nhằm tăng âm lượng như khi chơi đàn piano . trên đàn phong cầm , để tăng âm lượng , bạn sử dụng ống thổi tác động mạnh hơn hoặc nhanh hơn .'

In [ ]:
sample_encoded.tokens

In [ ]:
# Write to file